In [2]:
#Importando Bibliotecas
import pandas as pd
from selenium import webdriver
import win32com.client as win32


#função para tratamento de texto (valores retirados dos sites)
def transformar_texto(texto):
    return float(texto.replace('R$', '').replace('.', '').replace(',', '.'))

#lendo o arquivo com os links das lojas
produtos = pd.read_excel(r'Produtos.xlsx')
produtos = produtos.fillna('-')   #substitui os valores vazios pelo -

#rodar o código sem abrir o navegador.
driver = webdriver.Chrome()
driver.set_window_position(-10000,0)

enviar_email = False
desconto_minimo = 0.2

#utilizando o dataframe do arquivo para pegar o link e acessar o site.
for i, linha in produtos.iterrows():
    #pegar preço da Amazon
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_class_name('a-text-price').text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        try:
            preco_amazon = driver.find_element_by_css_selector('#olp_feature_div span.a-size-base').text
            preco_amazon = transformar_texto(preco_amazon)
        except:
            print('Produto {} não disponível na amazon'.format(linha['Link Produto']))
            preco_amazon = linha['Preço Original'] * 3
    

    #pegar preço da Americanas
    driver.get(linha['Lojas Americanas'])
    try:
        preco_lame = driver.find_element_by_class_name('src__BestPrice-sc-1jvw02c-5').text
        preco_lame = transformar_texto(preco_lame)
    except:
        print('Produto {} não disponível nas Lojas Americanas'.format(linha['Link Produto']))
        preco_lame = linha['Preço Original'] * 3
        
    #pegar preço da Magazzine Luiza
    driver.get(linha['Magazine Luiza'])
    try:
        preco_mglu = driver.find_element_by_class_name('price-template__text').text
        preco_mglu = transformar_texto(preco_mglu)
    except:
        print('Produto {} não disponível na Magazine Luiza'.format(linha['Link Produto']))
        preco_mglu = linha['Preço Original'] * 3
    
    #variável para poder fazer a comparação de preços
    preco_original = linha['Preço Original']
    
    #lista de preços para armazenar os preços coletados dos sites.
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lame, 'Lojas Americanas'), (preco_mglu, 'Magazine Luiza')]
    lista_precos.sort()
    
    #Inserindo os preços na coluna 'Preço Atual'
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0]
    
    #Inserindo a loja na coluna 'Local'
    produtos.loc[i, 'Local'] = lista_precos[0][1]
    
    if lista_precos[0][0] <= preco_original*(1-desconto_minimo):
        enviar_emailmail = True

#salvar o arquivo com os dados coletados dos sites.
produtos.to_excel('Produtos.xlsx')

#enviar email
outlook = win32.Dispatch('outlook.application')

if enviar_email:
    mail = outlook.CreateItem(0)
    mail.To = 'elivelton_sosk8@hotmail.com'
    mail.Subject = f'Produto(s) Encontrado(s) com mais de {desconto_minimo:.0%} de desconto'
    
    #filtrar a tabela de produtos
    tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_minimo), :]
    
    mail.HTMLBody = f'<p>Esses são os produtos com mais de {desconto_minimo:.0%} de desconto</p>{tabela_filtrada.to_html()}'
    
    mail.Send()

Produto iPhone 11 Apple 64GB Preto não disponível na Magazine Luiza
Produto Smart TV LED 50'' LG Ultra HD 4K Thinq AI não disponível nas Lojas Americanas
Produto Smart TV LED 50'' LG Ultra HD 4K Thinq AI não disponível na Magazine Luiza
